In [200]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 산불사건.csv 불러오기

In [201]:
f = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/산불발생일자19841201-20140619.csv')
f

,Unnamed: 0,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,0,201406191500,목,201406191800,300,북부,강원,인제,남,남전,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,1,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,2,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,3,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,4,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,10195,198412271900,목,198412272130,230,경북,경북,영천,임고,수성,...,-3.5,0.0,47.0,0.0,0.0,3.5,,NW,18,1.0
10196,10196,198412091300,일,198412091320,20,서울,서울,은평,불광,,...,13.3,0.0,34.0,0.0,0.0,3.0,,E,4,0.6
10197,10197,198412031150,월,198412031215,25,부산,부산,동래,명륜,,...,13.4,0.0,43.0,0.0,0.0,3.3,,W,10,2.7
10198,10198,198412021445,일,198412022145,700,경북,경북,청도,각남,옥산,...,10.6,0.0,19.0,0.0,0.0,4.0,,NNW,13,0.7


In [202]:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10200 non-null  int64  
 1   발생일시         10200 non-null  int64  
 2   발생일시(요일)     10200 non-null  object 
 3   진화일시(년월일시분)  10200 non-null  int64  
 4   진화소요시간(분)    10200 non-null  object 
 5   발생장소(관서)     10200 non-null  object 
 6   발생장소(시도)     10200 non-null  object 
 7   발생장소(시군구)    10200 non-null  object 
 8   발생장소(읍면동)    10200 non-null  object 
 9   발생장소(리)      10200 non-null  object 
 10  발생장소(지번)     10200 non-null  object 
 11  소유구분         10200 non-null  object 
 12  발생세부원인       10200 non-null  object 
 13  피해면적         10200 non-null  float64
 14  피해액          10200 non-null  int64  
 15  산불위험지수(평균)   10200 non-null  float64
 16  산불위험지수(최대)   10200 non-null  float64
 17  평균기온         10200 non-null  float64
 18  실효습도         10200 non-null  float64
 19  상대습도

In [203]:
f = f.drop('Unnamed: 0', axis=1)

In [204]:
f.columns

Index(['발생일시', '발생일시(요일)', '진화일시(년월일시분)', '진화소요시간(분)', '발생장소(관서)', '발생장소(시도)',
       '발생장소(시군구)', '발생장소(읍면동)', '발생장소(리)', '발생장소(지번)', '소유구분', '발생세부원인',
       '피해면적', '피해액', '산불위험지수(평균)', '산불위험지수(최대)', '평균기온', '실효습도', '상대습도',
       '최소습도', '최대풍속', '평균풍속', '최대풍향', '평균풍향', '강우결과일수', '최종강우량'],
      dtype='object')

In [205]:
f['발생장소(관서)']

0        북부
1        경남
2        강원
3        전북
4        북부
         ..
10195    경북
10196    서울
10197    부산
10198    경북
10199    전남
Name: 발생장소(관서), Length: 10200, dtype: object

# [산림청]산악기상 API

In [206]:
import requests

def forest_fires(local,time):
  API_KEY = "CJg6YBp/KNrl7YueJ48EwyvDpgka/lL0pUeuRvW84Aw+bDjTm0W1HVZSbhxk6mE4kU+jgLex72Up7mofH9hMUQ=="
  URL = 'http://apis.data.go.kr/1400377/mtweather/mountListSearch'

  params = {
      'serviceKey' : API_KEY,
      'pageNo' : '1', 
      'numOfRows' : '100', 
      '_type' : 'json', 
      'localArea' : local, 
      'tm' : time
  }
  response = requests.get(URL, params=params)
  datas = response.json()

  return datas

In [207]:
# # test
# forest_fires('16','201412020800')

# 학습용 데이터(샘플1)

In [208]:
f.columns

Index(['발생일시', '발생일시(요일)', '진화일시(년월일시분)', '진화소요시간(분)', '발생장소(관서)', '발생장소(시도)',
       '발생장소(시군구)', '발생장소(읍면동)', '발생장소(리)', '발생장소(지번)', '소유구분', '발생세부원인',
       '피해면적', '피해액', '산불위험지수(평균)', '산불위험지수(최대)', '평균기온', '실효습도', '상대습도',
       '최소습도', '최대풍속', '평균풍속', '최대풍향', '평균풍향', '강우결과일수', '최종강우량'],
      dtype='object')

In [209]:
f[:10]

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,201406191500,목,201406191800,300,북부,강원,인제,남,남전,산226-1,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,산123-1,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,산70,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,산65,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,산66,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
5,201406061450,금,201406061700,210,충북,충북,제천,봉양,미당,산77-1,...,28.7,0.0,28.0,0.0,7.7,2.9,E,E,3,2.0
6,201406020840,월,201406021505,625,경북,경북,포항 북,송라,중산,산97,...,22.7,0.0,75.0,0.0,3.7,1.9,S,N,5,4.9
7,201406012346,일,201406020730,744,북부,서울,은평,진관,,산34,...,23.5,0.0,29.0,0.0,0.9,1.3,ESE,W,6,1.5
8,201406012145,일,201406020150,405,강원,강원,횡성,갑천,상대,산90-1,...,25.4,0.0,51.0,0.0,1.9,1.1,N,SSW,6,5.0
9,201406010520,일,201406010930,410,경북,경북,예천,보문,오암,산17,...,19.2,0.0,14.0,0.0,1.7,1.0,NNW,SE,6,3.5


In [210]:
f[f['발생장소(시군구)']=='청도']

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
150,201404141417,월,201404141710,253,남부,경북,청도,운문,신원,산24-1,...,24.4,0.0,36.0,0.0,5.1,3.1,WNW,NNE,1,2.5
272,201403111520,화,201403111730,210,경북,경북,청도,매전,동산,산50,...,16.2,0.0,37.0,0.0,7.2,2.9,SW,SSE,10,2.0
364,201402161425,일,201402161545,120,경북,경북,청도,매전,동산,산118-1,...,11.5,0.0,86.0,0.0,6.1,2.4,W,E,15,1.5
407,201401211555,화,201401211650,55,경북,경북,청도,청도,거연,산308,...,2.8,0.0,23.0,0.0,4.7,3.5,NNW,NNW,13,3.5
601,201303251315,월,201303251420,105,경북,경북,청도,금천,박곡,산61,...,11.8,14.0,0.0,0.0,7.7,4.1,NNW,NNW,13,66.4
1142,201011301525,화,201011301610,45,남부,경북,청도,매전,금천,산88,...,17.0,47.6,46.0,0.0,4.2,1.3,WNW,SE,19,7.0
1262,201005161448,일,201005161625,137,경북,경북,청도,운문,오진,산22,...,28.1,48.7,17.0,0.0,3.3,2.5,WSW,S,6,1.0
1263,201005161448,일,201005161625,137,경북,경북,청도,운문,오진,산22,...,28.1,0.0,48.7,0.0,0.0,2.5,,S,6,0.5
1669,200905100400,일,200905101000,600,남부,경북,청도,운문,오진,산61,...,12.5,0.0,37.9,0.0,0.0,0.9,,NW,7,2.0
1720,200904191905,일,200904192230,325,경북,경북,청도,금천,박곡,산161,...,22.4,0.0,44.4,0.0,0.0,3.0,,S,5,6.0


In [211]:
# forest_fires('10','201406191500')

산악기상API 말고 기상청에서 파일 다운 받기로함.

In [212]:
d=pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/SURFACE_AWS_813_MI_2014-04_2014-04_2018.csv', encoding="cp949")
d

,지점,일시,기온(°C),1분 강수량(mm),강수유무(유무),풍향(deg),풍속(m/s),현지기압(hPa),해면기압(hPa),습도(%),일사(MJ/m^2),일조(Sec)
0,813,2014-04-01 00:01,9.0,0.0,0,304.0,0.5,NaN,NaN,85.1,0,0
1,813,2014-04-01 00:02,9.0,0.0,0,306.1,0.6,NaN,NaN,85.2,0,0
2,813,2014-04-01 00:03,8.9,0.0,0,322.5,0.7,NaN,NaN,85.1,0,0
3,813,2014-04-01 00:04,8.9,0.0,0,334.1,0.6,NaN,NaN,85.1,0,0
4,813,2014-04-01 00:05,8.9,0.0,0,350.2,0.6,NaN,NaN,85.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43135,813,2014-04-30 23:56,13.5,0.0,10,240.2,1.1,NaN,NaN,93.2,0,0
43136,813,2014-04-30 23:57,13.5,0.0,10,239.7,1.1,NaN,NaN,93.2,0,0
43137,813,2014-04-30 23:58,13.5,0.0,10,232.4,1.2,NaN,NaN,93.2,0,0
43138,813,2014-04-30 23:59,13.5,0.0,10,232.2,1.1,NaN,NaN,93.2,0,0


In [213]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43140 entries, 0 to 43139
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지점          43140 non-null  int64  
 1   일시          43140 non-null  object 
 2   기온(°C)      43140 non-null  float64
 3   1분 강수량(mm)  43076 non-null  float64
 4   강수유무(유무)    43140 non-null  int64  
 5   풍향(deg)     43140 non-null  float64
 6   풍속(m/s)     43140 non-null  float64
 7   현지기압(hPa)   0 non-null      float64
 8   해면기압(hPa)   0 non-null      float64
 9   습도(%)       43140 non-null  float64
 10  일사(MJ/m^2)  43140 non-null  int64  
 11  일조(Sec)     43140 non-null  int64  
dtypes: float64(7), int64(4), object(1)
memory usage: 3.9+ MB


In [214]:
d['일시']

0        2014-04-01 00:01
1        2014-04-01 00:02
2        2014-04-01 00:03
3        2014-04-01 00:04
4        2014-04-01 00:05
               ...       
43135    2014-04-30 23:56
43136    2014-04-30 23:57
43137    2014-04-30 23:58
43138    2014-04-30 23:59
43139    2014-05-01 00:00
Name: 일시, Length: 43140, dtype: object

In [215]:
for dd in d['일시']:
  
    if dd[:10] == '2014-04-14':
      dfdf = d.loc[(d['일시'] == dd), :]
    else:
      pass

dfdf

,지점,일시,기온(°C),1분 강수량(mm),강수유무(유무),풍향(deg),풍속(m/s),현지기압(hPa),해면기압(hPa),습도(%),일사(MJ/m^2),일조(Sec)
20156,813,2014-04-14 23:59,8.8,0.0,0,287.1,1.0,NaN,NaN,62.2,0,0


## 날짜 파싱

In [216]:
d['일시'] = pd.to_datetime(d['일시'])

In [217]:
# d[d['일시'].dt.day == 14].to_csv('2014-04-14,경북,청도군,분 데이터.csv', index=False)

In [218]:
pp = f[f['발생장소(시군구)']=='청도']
pp

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
150,201404141417,월,201404141710,253,남부,경북,청도,운문,신원,산24-1,...,24.4,0.0,36.0,0.0,5.1,3.1,WNW,NNE,1,2.5
272,201403111520,화,201403111730,210,경북,경북,청도,매전,동산,산50,...,16.2,0.0,37.0,0.0,7.2,2.9,SW,SSE,10,2.0
364,201402161425,일,201402161545,120,경북,경북,청도,매전,동산,산118-1,...,11.5,0.0,86.0,0.0,6.1,2.4,W,E,15,1.5
407,201401211555,화,201401211650,55,경북,경북,청도,청도,거연,산308,...,2.8,0.0,23.0,0.0,4.7,3.5,NNW,NNW,13,3.5
601,201303251315,월,201303251420,105,경북,경북,청도,금천,박곡,산61,...,11.8,14.0,0.0,0.0,7.7,4.1,NNW,NNW,13,66.4
1142,201011301525,화,201011301610,45,남부,경북,청도,매전,금천,산88,...,17.0,47.6,46.0,0.0,4.2,1.3,WNW,SE,19,7.0
1262,201005161448,일,201005161625,137,경북,경북,청도,운문,오진,산22,...,28.1,48.7,17.0,0.0,3.3,2.5,WSW,S,6,1.0
1263,201005161448,일,201005161625,137,경북,경북,청도,운문,오진,산22,...,28.1,0.0,48.7,0.0,0.0,2.5,,S,6,0.5
1669,200905100400,일,200905101000,600,남부,경북,청도,운문,오진,산61,...,12.5,0.0,37.9,0.0,0.0,0.9,,NW,7,2.0
1720,200904191905,일,200904192230,325,경북,경북,청도,금천,박곡,산161,...,22.4,0.0,44.4,0.0,0.0,3.0,,S,5,6.0


In [219]:
# pp.loc[150,:].to_csv('2014-04-14,경북,청도군,사건데이터.csv')

In [220]:
pp.loc[150,:]

발생일시           201404141417
발생일시(요일)                  월
진화일시(년월일시분)    201404141710
진화소요시간(분)               253
발생장소(관서)                 남부
발생장소(시도)                 경북
발생장소(시군구)                청도
발생장소(읍면동)                운문
발생장소(리)                  신원
발생장소(지번)              산24-1
소유구분                       
발생세부원인             입산자실화 추정
피해면적                    0.8
피해액                       0
산불위험지수(평균)              0.0
산불위험지수(최대)              0.0
평균기온                   24.4
실효습도                    0.0
상대습도                   36.0
최소습도                    0.0
최대풍속                    5.1
평균풍속                    3.1
최대풍향                    WNW
평균풍향                    NNE
강우결과일수                    1
최종강우량                   2.5
Name: 150, dtype: object

# 학습용 데이터(샘플2)

- 제일 오래된 날짜부터 30개씩 자르되 이름순으로 가져가기
- 난 4번째여서 90~120 번째의 사건들을 맡으면됨.
- 각 사건들의 발생일시가 기준일이 되고 해당 기준일의 2주전부터 기준일시까지의 기상데이터를 수집
- 기상데이터 존재유무 표시해주는 컬럼추가

In [221]:
f

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,201406191500,목,201406191800,300,북부,강원,인제,남,남전,산226-1,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,산123-1,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,산70,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,산65,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,산66,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,198412271900,목,198412272130,230,경북,경북,영천,임고,수성,산185,...,-3.5,0.0,47.0,0.0,0.0,3.5,,NW,18,1.0
10196,198412091300,일,198412091320,20,서울,서울,은평,불광,,산43-1,...,13.3,0.0,34.0,0.0,0.0,3.0,,E,4,0.6
10197,198412031150,월,198412031215,25,부산,부산,동래,명륜,,산50-7,...,13.4,0.0,43.0,0.0,0.0,3.3,,W,10,2.7
10198,198412021445,일,198412022145,700,경북,경북,청도,각남,옥산,산350,...,10.6,0.0,19.0,0.0,0.0,4.0,,NNW,13,0.7


In [222]:
data_df = f.sort_values(by='발생일시')[90:120]
data_df

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
10109,198604081420,화,198604081510,50,강원,강원,원주,흥업,대안,산38-1,...,18.6,0.0,21.0,0.0,0.0,3.7,,WNW,17,2.5
10108,198604081510,화,198604082030,520,강원,강원,횡성,갑천,구방,산5,...,19.6,0.0,19.0,0.0,0.0,4.0,,SW,9,0.8
10107,198604081530,화,198604081647,117,대전,대전,동,하소,,산51,...,19.2,0.0,21.0,0.0,0.0,8.0,,W,17,1.6
10106,198604081600,화,198604081650,50,강원,강원,춘천,동내,학곡,산103,...,16.3,0.0,32.0,0.0,0.0,8.0,,SSW,9,0.7
10105,198604111245,금,198604111500,215,강원,강원,평창,도암,유천,산371-1,...,8.5,0.0,28.0,0.0,0.0,2.7,,W,12,1.3
10104,198604111400,금,198604111510,110,충남,충남,공주,계룡,향지,산11-12,...,13.0,0.0,11.0,0.0,0.0,8.3,,NW,1,0.5
10103,198604131330,일,198604131417,47,충남,충남,예산,예산,향천,산28-1,...,19.1,0.0,30.0,0.0,0.0,3.0,,SW,14,0.5
10102,198604131515,일,198604131525,10,강원,강원,춘천,삼천,,산5-3,...,18.7,0.0,26.0,0.0,0.0,7.7,,SW,14,0.7
10101,198604141245,월,198604141445,200,강원,강원,정선,신동,조동,산94,...,2.4,0.0,67.0,0.0,0.0,0.2,,S,23,2.0
10100,198604170530,목,198604170700,130,강원,강원,춘천,북산,청평,산192-1,...,1.0,0.0,80.0,0.0,0.0,0.0,,정온,18,0.7


In [223]:
data_df.columns

Index(['발생일시', '발생일시(요일)', '진화일시(년월일시분)', '진화소요시간(분)', '발생장소(관서)', '발생장소(시도)',
       '발생장소(시군구)', '발생장소(읍면동)', '발생장소(리)', '발생장소(지번)', '소유구분', '발생세부원인',
       '피해면적', '피해액', '산불위험지수(평균)', '산불위험지수(최대)', '평균기온', '실효습도', '상대습도',
       '최소습도', '최대풍속', '평균풍속', '최대풍향', '평균풍향', '강우결과일수', '최종강우량'],
      dtype='object')

In [224]:
data_df = data_df.reset_index(drop=True)

In [225]:
data_df['기상데이터_존재유무'] = 'Y'
data_df.loc[[4,6,8,15,19,20,21], '기상데이터_존재유무'] = 'N'
data_df

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량,기상데이터_존재유무
0,198604081420,화,198604081510,50,강원,강원,원주,흥업,대안,산38-1,...,0.0,21.0,0.0,0.0,3.7,,WNW,17,2.5,Y
1,198604081510,화,198604082030,520,강원,강원,횡성,갑천,구방,산5,...,0.0,19.0,0.0,0.0,4.0,,SW,9,0.8,Y
2,198604081530,화,198604081647,117,대전,대전,동,하소,,산51,...,0.0,21.0,0.0,0.0,8.0,,W,17,1.6,Y
3,198604081600,화,198604081650,50,강원,강원,춘천,동내,학곡,산103,...,0.0,32.0,0.0,0.0,8.0,,SSW,9,0.7,Y
4,198604111245,금,198604111500,215,강원,강원,평창,도암,유천,산371-1,...,0.0,28.0,0.0,0.0,2.7,,W,12,1.3,N
5,198604111400,금,198604111510,110,충남,충남,공주,계룡,향지,산11-12,...,0.0,11.0,0.0,0.0,8.3,,NW,1,0.5,Y
6,198604131330,일,198604131417,47,충남,충남,예산,예산,향천,산28-1,...,0.0,30.0,0.0,0.0,3.0,,SW,14,0.5,N
7,198604131515,일,198604131525,10,강원,강원,춘천,삼천,,산5-3,...,0.0,26.0,0.0,0.0,7.7,,SW,14,0.7,Y
8,198604141245,월,198604141445,200,강원,강원,정선,신동,조동,산94,...,0.0,67.0,0.0,0.0,0.2,,S,23,2.0,N
9,198604170530,목,198604170700,130,강원,강원,춘천,북산,청평,산192-1,...,0.0,80.0,0.0,0.0,0.0,,정온,18,0.7,Y


In [226]:
# 저장 & 다운
from google.colab import files

# data_df.to_csv('event_data.csv', index=False)
# files.download('event_data.csv')

## 기상청 API에서 데이터 수집

API_KEY를 이요해 자동으로 데이터를 수집해주고 데이터 요청시에 필수 파라미터인값들을 forest_fires2 함수의 파라미터로 하여 일일히 입력하지 않아도 밑에 5개 파라미터만 입력해주면 수집할때마다 API키와 URL, 데이터타입 등등을 같이 입력해줄필요가 없게 된다.
  - 시작일
  - 시작시간
  - 종료일
  - 종료시간
  - 기상관측 지점번호  
***
  - 지역명  

+추가로 지역명도 있으면 데이터를 알아보기 쉬울거같아 함수를 사용할때 같이 입력받아서 데이터를 저장하는걸로 변경하였다.

In [227]:
import re

def forest_fires2(start,start_time,end,end_time,local,localname):
  API_KEY = "CJg6YBp/KNrl7YueJ48EwyvDpgka/lL0pUeuRvW84Aw+bDjTm0W1HVZSbhxk6mE4kU+jgLex72Up7mofH9hMUQ=="
  URL = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

  params = {
      'serviceKey' : API_KEY,
      'pageNo' : '1', 
      'numOfRows' : '900', 
      'dataType' : 'json',
      'dataCd' : 'ASOS', 
      'dateCd' : 'HR',
      'startDt' : start, 
      'startHh' : start_time, 
      'endDt' : end, 
      'endHh' : end_time, 
      'stnIds' : local
  }

  response = requests.get(URL, params=params)
  datas = response.json()

  full_data = []

  for a in datas['response']['body']['items']['item']:
    tm = a['tm']
    ws = a['ws']
    wd = a['wd']
    hm = a['hm']
    ta = a['ta']
    rn = a['rn']

    # try:
    #   day_pattern = end
    #   day = tm.split('-')[0] + tm.split('-')[1] + tm.split('-')[2][:2]
    #   re.match(day_pattern, day).group()   # 산불 발생일이 아니면 에러 발생 1

    #   time_pattern = end_time
    #   time = tm.split(' ')[1][:2]
    #   re.match(time_pattern, time).group()  # 산불 발생 시간이 아니면 에러 발생 2

    #   label = 1 # 사건 발생날, 해당 시간을 1로 라벨링
    # except:
    #   label = 0  # 사건과 무관한날, 시간은 0으로 라벨링

    # 지점번호, 지역명
    local = local
    local_name = localname

    #data = [local_name, local, tm, ws, wd, hm, ta, rn, label]
    data = [local_name, local, tm, ws, wd, hm, ta, rn]

    full_data.append(data)

  return full_data

In [228]:
# test
a = '1986-03-25 15:00'
aaa = a.split('-')[0] + a.split('-')[1] + a.split('-')[2][:2]
aaa

'19860325'

In [229]:
# test
a = '1986-03-25 15:00'
a.split(' ')[1][:2]

'15'

In [230]:
# test
forest_fires2('19860405','15','19860419','15','101','춘천')

[['춘천', '101', '1986-04-05 15:00', '10.0', '180', '29', '16.1', ''],
 ['춘천', '101', '1986-04-05 16:00', '8.3', '360', '', '15.6', ''],
 ['춘천', '101', '1986-04-05 17:00', '9.0', '360', '', '14.8', ''],
 ['춘천', '101', '1986-04-05 18:00', '6.3', '360', '29', '14.2', ''],
 ['춘천', '101', '1986-04-05 19:00', '5.7', '180', '', '12.6', ''],
 ['춘천', '101', '1986-04-05 20:00', '6.3', '180', '', '11.3', ''],
 ['춘천', '101', '1986-04-05 21:00', '2.3', '160', '42', '10.4', ''],
 ['춘천', '101', '1986-04-05 22:00', '1.5', '180', '', '9.2', ''],
 ['춘천', '101', '1986-04-05 23:00', '2.5', '180', '', '8.8', ''],
 ['춘천', '101', '1986-04-06 00:00', '1.3', '180', '60', '8.3', ''],
 ['춘천', '101', '1986-04-06 01:00', '1.3', '140', '', '', ''],
 ['춘천', '101', '1986-04-06 02:00', '0.5', '140', '', '', ''],
 ['춘천', '101', '1986-04-06 03:00', '0.0', '0', '79', '4.5', ''],
 ['춘천', '101', '1986-04-06 04:00', '0.0', '0', '', '2.5', ''],
 ['춘천', '101', '1986-04-06 05:00', '0.0', '0', '', '1.6', ''],
 ['춘천', '101', '198

In [231]:
# # 노가다 -> 30개 사건 기준 2주전ㅂ터 사건 당일 시간까지의 기상데이터 수집
# # 시간은 30 이하는 버리고 30분 이상은 올려서 ex) 14시 20분에 사건발생시 14시로 , 14시 40분에 사건발생시 15시로 
# fire1 = forest_fires2('19860325','14','19860408','14','114','원주')
# fire2 = forest_fires2('19860325','15','19860408','15','114','원주')
# fire3 = forest_fires2('19860325','16','19860408','16','133','대전')
# fire4 = forest_fires2('19860325','16','19860408','16','101','춘천')
# # fire5 = forest_fires2('19860328','13','19860411','13','217','평창') -> {'response': {'header': {'resultCode': '03', 'resultMsg': 'NO_DATA'}}}, 2011부터 데이터 있음.
# fire6 = forest_fires2('19860328','14','19860411','14','236','공주')
# # fire7 = forest_fires2('19860330','14','19860413','14','177','예산') -> {'response': {'header': {'resultCode': '03', 'resultMsg': 'NO_DATA'}}} , 2014까지 데이터가 하나도 없음.
# fire8 = forest_fires2('19860330','15','19860413','15','101','춘천')
# # fire9 = forest_fires2('19860331','13','19860414','13','217','정선') ->  2011부터 데이터 있음.
# fire10 = forest_fires2('19860403','06','19860417','06','101','춘천')
# fire11 = forest_fires2('19860403','12','19860417','12','211','양구')
# fire12 = forest_fires2('19860403','14','19860417','14','105','강릉')
# fire13 = forest_fires2('19860404','13','19860418','13','101','춘천')
# fire14 = forest_fires2('19860405','15','19860419','15','101','춘천')
# fire15 = forest_fires2('19860405','18','19860419','18','212','홍천')
# # fire16 = forest_fires2('19860406','17','19860420','17','217','정선') -> 2011부터 데이터 있음.
# fire17 = forest_fires2('19860408','12','19860422','12','114','원주')
# fire18 = forest_fires2('19860410','11','19860424','11','211','인제')
# fire19 = forest_fires2('19860410','12','19860424','12','101','춘천')
# # fire20 = forest_fires2('19860424','13','19860508','13','217','정선') -> 2011부터 데이터 있음.
# # fire21 = forest_fires2('19861104','14','19861118','14','106','동해') -> 1992년 부터 있음.
# # fire22 = forest_fires2('19861116','12','19861130','12','106','삼척') -> 1992년 부터 있음.
# fire23 = forest_fires2('19870206','16','19870220','16','101','춘천')
# fire24 = forest_fires2('19870208','13','19870222','13','101','춘천')
# fire25 = forest_fires2('19870301','13','19870315','13','143','달서')
# fire26 = forest_fires2('19870302','15','19870316','15','212','홍천')
# fire27 = forest_fires2('19870315','14','19870329','14','101','춘천')
# fire28 = forest_fires2('19870318','18','19870401','18','101','춘천')
# fire29 = forest_fires2('19870321','11','19870404','11','143','대구')
# fire30 = forest_fires2('19870321','14','19870404','14','143','대구')

# # 데이터 없는 애들
# fire5 = None
# fire7 = None
# fire9 = None
# fire16 = None
# fire20 = None
# fire21 = None
# fire22 = None

In [232]:
# 데이터 프레임 만드는 함수
def data_frame(fire):

  # df = pd.DataFrame(fire, columns=['지역','지점번호','일시','풍속','풍향','상대습도','기온','강수량','라벨'])
  df = pd.DataFrame(fire, columns=['지역','지점번호','일시','풍속','풍향','습도','기온','강수량'])

  return df

### 1번

In [233]:
# fire1_df = data_frame(fire1)
# fire1_df

In [234]:
# fire1_df[(fire1_df['강수량'] != '') & (fire1_df['강수량'] > '0.0')]

In [235]:
# fire1_df['강우_후_경과시간(hour)'] = None
# fire1_df.loc[:111,'강우_후_경과시간(hour)'] = None
# fire1_df.loc[112:115,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire1_df.loc[116:,'강우_후_경과시간(hour)'])):
#   fire1_df.loc[116+i,'강우_후_경과시간(hour)'] = i+1

# fire1_df.loc[116:,'강우_후_경과시간(hour)']

In [236]:
# # 저장
# fire1_df.to_csv('event_01.csv',index=False)
# files.download('event_01.csv')

### 2번

In [237]:
# fire2_df = data_frame(fire2)
# fire2_df[(fire2_df['강수량'] != '') & (fire2_df['강수량'] > '0.0')]

In [238]:
# fire2_df['강우_후_경과시간(hour)'] = None
# fire2_df.loc[:110,'강우_후_경과시간(hour)'] = None
# fire2_df.loc[111:114,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire2_df.loc[115:,'강우_후_경과시간(hour)'])):
#   fire2_df.loc[115+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire2_df.to_csv('event_02.csv',index=False)
# files.download('event_02.csv')

### 3번

In [239]:
# fire3_df = data_frame(fire3)
# fire3_df[(fire3_df['강수량'] != '') & (fire3_df['강수량'] > '0.0')]

In [240]:
# fire3_df['강우_후_경과시간(hour)'] = None

# # 저장
# fire3_df.to_csv('event_03.csv',index=False)
# files.download('event_03.csv')

### 4번

In [241]:
# fire4_df = data_frame(fire4)
# fire4_df[(fire4_df['강수량'] != '') & (fire4_df['강수량'] > '0.0')]

In [242]:
# fire4_df['강우_후_경과시간(hour)'] = None
# fire4_df.loc[:103,'강우_후_경과시간(hour)'] = None
# fire4_df.loc[104:110,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire4_df.loc[111:,'강우_후_경과시간(hour)'])):
#   fire4_df.loc[111+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire4_df.to_csv('event_04.csv',index=False)
# files.download('event_04.csv')

### 6번

In [243]:
# fire6_df = data_frame(fire6)
# fire6_df[(fire6_df['강수량'] != '') & (fire6_df['강수량'] > '0.0')]

In [244]:
# fire6_df['강우_후_경과시간(hour)'] = None
# fire6_df.loc[:281,'강우_후_경과시간(hour)'] = None
# fire6_df.loc[282,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire6_df.loc[283:300,'강우_후_경과시간(hour)'])):
#   fire6_df.loc[283+i,'강우_후_경과시간(hour)'] = i+1


# fire6_df.loc[301,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire6_df.loc[302:,'강우_후_경과시간(hour)'])):
#   fire6_df.loc[302+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire6_df.to_csv('event_06.csv',index=False)
# files.download('event_06.csv')

### 8번

In [245]:
# fire8_df = data_frame(fire8)
# fire8_df[(fire8_df['강수량'] != '') & (fire8_df['강수량'] > '0.0')]

In [246]:
# fire8_df['강우_후_경과시간(hour)'] = None

# # 저장
# fire8_df.to_csv('event_08.csv',index=False)
# files.download('event_08.csv')

### 10번

In [247]:
# fire10_df = data_frame(fire10)
# fire10_df[(fire10_df['강수량'] != '') & (fire10_df['강수량'] > '0.0')]

In [248]:
# fire10_df['강우_후_경과시간(hour)'] = None
# fire10_df.loc[:285,'강우_후_경과시간(hour)'] = None
# fire10_df.loc[286,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire10_df.loc[287:,'강우_후_경과시간(hour)'])):
#   fire10_df.loc[287+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire10_df.to_csv('event_10.csv',index=False)
# files.download('event_10.csv')

### 11번

In [249]:
# fire11_df = data_frame(fire11)
# fire11_df[(fire11_df['강수량'] != '') & (fire11_df['강수량'] > '0.0')]

In [250]:
# fire11_df['강우_후_경과시간(hour)'] = None

# # 저장
# fire11_df.to_csv('event_11.csv',index=False)
# files.download('event_11.csv')

### 12번

In [251]:
# fire12_df = data_frame(fire12)
# fire12_df[(fire12_df['강수량'] != '') & (fire12_df['강수량'] > '0.0')]

In [252]:
# fire12_df['강우_후_경과시간(hour)'] = None
# fire12_df.loc[:273,'강우_후_경과시간(hour)'] = None
# fire12_df.loc[274:279,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire12_df.loc[280:,'강우_후_경과시간(hour)'])):
#   fire12_df.loc[280+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire12_df.to_csv('event_12.csv',index=False)
# files.download('event_12.csv')

### 13번

In [253]:
# fire13_df = data_frame(fire13)
# fire13_df[(fire13_df['강수량'] != '') & (fire13_df['강수량'] > '0.0')]

In [254]:
# fire13_df['강우_후_경과시간(hour)'] = None
# fire13_df.loc[:254,'강우_후_경과시간(hour)'] = None
# fire13_df.loc[255,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire13_df.loc[256:,'강우_후_경과시간(hour)'])):
#   fire13_df.loc[256+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire13_df.to_csv('event_13.csv',index=False)
# files.download('event_13.csv')

### 14번

In [255]:
# fire14_df = data_frame(fire14)
# fire14_df[(fire14_df['강수량'] != '') & (fire14_df['강수량'] > '0.0')]

In [256]:
# fire14_df['강우_후_경과시간(hour)'] = None
# fire14_df.loc[:228,'강우_후_경과시간(hour)'] = None
# fire14_df.loc[229,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire14_df.loc[230:,'강우_후_경과시간(hour)'])):
#   fire14_df.loc[230+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire14_df.to_csv('event_14.csv',index=False)
# files.download('event_14.csv')

### 15번

In [257]:
# fire15_df = data_frame(fire15)
# fire15_df[(fire15_df['강수량'] != '') & (fire15_df['강수량'] > '0.0')]

In [258]:
# fire15_df['강우_후_경과시간(hour)'] = None
# fire15_df.loc[:224,'강우_후_경과시간(hour)'] = None
# fire15_df.loc[225:229,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire15_df.loc[230:,'강우_후_경과시간(hour)'])):
#   fire15_df.loc[230+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire15_df.to_csv('event_15.csv',index=False)
# files.download('event_15.csv')

### 17번

In [259]:
# fire17_df = data_frame(fire17)
# fire17_df[(fire17_df['강수량'] != '') & (fire17_df['강수량'] > '0.0')]

In [260]:
# fire17_df.loc[156:162,'강수량']

In [261]:
# fire17_df['강우_후_경과시간(hour)'] = None
# fire17_df.loc[:40,'강우_후_경과시간(hour)'] = None
# fire17_df.loc[41,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire17_df.loc[42:155,'강우_후_경과시간(hour)'])):
#   fire17_df.loc[42+i,'강우_후_경과시간(hour)'] = i+1

# fire17_df.loc[156:160,'강우_후_경과시간(hour)'] = 0

# for i in range(1):
#   fire17_df.loc[161+i,'강우_후_경과시간(hour)'] = i+1

# fire17_df.loc[162,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire17_df.loc[163:,'강우_후_경과시간(hour)'])):
#   fire17_df.loc[163+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire17_df.to_csv('event_17.csv',index=False)
# files.download('event_17.csv')

### 18번

In [262]:
# fire18_df = data_frame(fire18)
# fire18_df[(fire18_df['강수량'] != '') & (fire18_df['강수량'] > '0.0')]

In [263]:
# fire18_df['강우_후_경과시간(hour)'] = None

# # 저장
# fire18_df.to_csv('event_18.csv',index=False)
# files.download('event_18.csv')

### 19번

In [264]:
# fire19_df = data_frame(fire19)
# fire19_df[(fire19_df['강수량'] != '') & (fire19_df['강수량'] > '0.0')]

In [265]:
# fire19_df.loc[112:120,'강수량']

In [266]:
# fire19_df['강우_후_경과시간(hour)'] = None
# fire19_df.loc[:111,'강우_후_경과시간(hour)'] = None
# fire19_df.loc[112,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire19_df.loc[113:,'강우_후_경과시간(hour)'])):
#   fire19_df.loc[113+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire19_df.to_csv('event_19.csv',index=False)
# files.download('event_19.csv')

In [267]:
# fire19_df.loc[110:120,:]

### 23번

In [268]:
# fire23_df = data_frame(fire23)
# fire23_df[(fire23_df['강수량'] != '') & (fire23_df['강수량'] > '0.0')]

In [269]:
# fire23_df.loc[107:116,'강수량']

In [270]:
# fire23_df['강우_후_경과시간(hour)'] = None
# fire23_df.loc[:91,'강우_후_경과시간(hour)'] = None
# fire23_df.loc[92:107,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire23_df.loc[108:115,'강우_후_경과시간(hour)'])):
#   fire23_df.loc[108+i,'강우_후_경과시간(hour)'] = i+1

# fire23_df.loc[116:125,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire23_df.loc[126:133,'강우_후_경과시간(hour)'])):
#   fire23_df.loc[126+i,'강우_후_경과시간(hour)'] = i+1

# fire23_df.loc[134,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire23_df.loc[135:172,'강우_후_경과시간(hour)'])):
#   fire23_df.loc[135+i,'강우_후_경과시간(hour)'] = i+1

# fire23_df.loc[173:179,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire23_df.loc[180:,'강우_후_경과시간(hour)'])):
#   fire23_df.loc[180+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire23_df.to_csv('event_23.csv',index=False)
# files.download('event_23.csv')

### 24번

In [271]:
# fire24_df = data_frame(fire24)
# fire24_df[(fire24_df['강수량'] != '') & (fire24_df['강수량'] > '0.0')]

In [272]:
# fire24_df.loc[62:71,'강수량']

In [273]:
# fire24_df['강우_후_경과시간(hour)'] = None
# fire24_df.loc[:46,'강우_후_경과시간(hour)'] = None
# fire24_df.loc[47:62,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire24_df.loc[63:70,'강우_후_경과시간(hour)'])):
#   fire24_df.loc[63+i,'강우_후_경과시간(hour)'] = i+1

# fire24_df.loc[71:80,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire24_df.loc[81:88,'강우_후_경과시간(hour)'])):
#   fire24_df.loc[81+i,'강우_후_경과시간(hour)'] = i+1

# fire24_df.loc[89,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire24_df.loc[90:127,'강우_후_경과시간(hour)'])):
#   fire24_df.loc[90+i,'강우_후_경과시간(hour)'] = i+1

# fire24_df.loc[128:134,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire24_df.loc[135:,'강우_후_경과시간(hour)'])):
#   fire24_df.loc[135+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire24_df.to_csv('event_24.csv',index=False)
# files.download('event_24.csv')

### 25번

In [274]:
# fire25_df = data_frame(fire25)
# fire25_df[(fire25_df['강수량'] != '') & (fire25_df['강수량'] > '0.0')]

In [275]:
# fire25_df.loc[140:146,'강수량']

In [276]:
# fire25_df['강우_후_경과시간(hour)'] = None
# fire25_df.loc[:136,'강우_후_경과시간(hour)'] = None
# fire25_df.loc[137:146,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire25_df.loc[147:283,'강우_후_경과시간(hour)'])):
#   fire25_df.loc[147+i,'강우_후_경과시간(hour)'] = i+1

# fire25_df.loc[284,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire25_df.loc[285:,'강우_후_경과시간(hour)'])):
#   fire25_df.loc[285+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire25_df.to_csv('event_25.csv',index=False)
# files.download('event_25.csv')

### 26번

In [277]:
# fire26_df = data_frame(fire26)
# fire26_df[(fire26_df['강수량'] != '') & (fire26_df['강수량'] > '0.0')]

In [278]:
# fire26_df.loc[45:54,'강수량']

In [279]:
# fire26_df['강우_후_경과시간(hour)'] = None
# fire26_df.loc[:47,'강우_후_경과시간(hour)'] = None
# fire26_df.loc[48:57,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire26_df.loc[58:197,'강우_후_경과시간(hour)'])):
#   fire26_df.loc[58+i,'강우_후_경과시간(hour)'] = i+1

# fire26_df.loc[198:201,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire26_df.loc[202:,'강우_후_경과시간(hour)'])):
#   fire26_df.loc[202+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire26_df.to_csv('event_26.csv',index=False)
# files.download('event_26.csv')

### 27번

In [280]:
# fire27_df = data_frame(fire27)
# fire27_df[(fire27_df['강수량'] != '') & (fire27_df['강수량'] > '0.0')]

In [281]:
# fire27_df['강우_후_경과시간(hour)'] = None
# fire27_df.loc[:54,'강우_후_경과시간(hour)'] = None

# fire27_df.loc[55,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire27_df.loc[56:90,'강우_후_경과시간(hour)'])):
#   fire27_df.loc[56+i,'강우_후_경과시간(hour)'] = i+1

# fire27_df.loc[91:100,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire27_df.loc[101:168,'강우_후_경과시간(hour)'])):
#   fire27_df.loc[101+i,'강우_후_경과시간(hour)'] = i+1

# fire27_df.loc[169:172,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire27_df.loc[173:204,'강우_후_경과시간(hour)'])):
#   fire27_df.loc[173+i,'강우_후_경과시간(hour)'] = i+1

# fire27_df.loc[205:214,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire27_df.loc[215:222,'강우_후_경과시간(hour)'])):
#   fire27_df.loc[215+i,'강우_후_경과시간(hour)'] = i+1

# fire27_df.loc[223,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire27_df.loc[224:,'강우_후_경과시간(hour)'])):
#   fire27_df.loc[224+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire27_df.to_csv('event_27.csv',index=False)
# files.download('event_27.csv')

### 28번

In [282]:
# fire28_df = data_frame(fire28)
# fire28_df[(fire28_df['강수량'] != '') & (fire28_df['강수량'] > '0.0')]

In [283]:
# fire28_df.loc[138:147,'강수량']

In [284]:
# fire28_df['강우_후_경과시간(hour)'] = None
# fire28_df.loc[:14,'강우_후_경과시간(hour)'] = None

# fire28_df.loc[15:24,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire28_df.loc[25:92,'강우_후_경과시간(hour)'])):
#   fire28_df.loc[25+i,'강우_후_경과시간(hour)'] = i+1

# fire28_df.loc[93:96,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire28_df.loc[97:128,'강우_후_경과시간(hour)'])):
#   fire28_df.loc[97+i,'강우_후_경과시간(hour)'] = i+1

# fire28_df.loc[129:138,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire28_df.loc[139:146,'강우_후_경과시간(hour)'])):
#   fire28_df.loc[139+i,'강우_후_경과시간(hour)'] = i+1

# fire28_df.loc[147,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire28_df.loc[148:290,'강우_후_경과시간(hour)'])):
#   fire28_df.loc[148+i,'강우_후_경과시간(hour)'] = i+1

# fire28_df.loc[291,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire28_df.loc[292:,'강우_후_경과시간(hour)'])):
#   fire28_df.loc[292+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire28_df.to_csv('event_28.csv',index=False)
# files.download('event_28.csv')

### 29번

In [285]:
# fire29_df = data_frame(fire29)
# fire29_df[(fire29_df['강수량'] != '') & (fire29_df['강수량'] > '0.0')]

In [286]:
# fire29_df['강우_후_경과시간(hour)'] = None
# fire29_df.loc[:36,'강우_후_경과시간(hour)'] = None

# fire29_df.loc[37:73,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire29_df.loc[74:,'강우_후_경과시간(hour)'])):
#   fire29_df.loc[74+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire29_df.to_csv('event_29.csv',index=False)
# files.download('event_29.csv')

### 30번

In [287]:
# fire30_df = data_frame(fire30)
# fire30_df[(fire30_df['강수량'] != '') & (fire30_df['강수량'] > '0.0')]

In [288]:
# fire30_df['강우_후_경과시간(hour)'] = None
# fire30_df.loc[:33,'강우_후_경과시간(hour)'] = None

# fire30_df.loc[34:70,'강우_후_경과시간(hour)'] = 0

# for i in range(len(fire30_df.loc[71:,'강우_후_경과시간(hour)'])):
#   fire30_df.loc[71+i,'강우_후_경과시간(hour)'] = i+1

# # 저장
# fire30_df.to_csv('event_30.csv',index=False)
# files.download('event_30.csv')

# ASOS 종관기상관측 각 지점별 위도,경도 수집
- "https://minwon.kma.go.kr/main/obvStn.do"
- 위 사이트에서 크롤링하여 위,경도 정보 가져오기

In [289]:
URL = 'https://minwon.kma.go.kr/main/obvStn.do'

res = requests.get(URL)
soup = BeautifulSoup(res.content, 'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>관측 지점 정보</title>
<!-- <link rel="stylesheet" type="text/css" href="/css/kma2012.css" /> -->
<link href="/css/style.css" rel="stylesheet" type="text/css"/>
<link href="/js/jquery/jquery-ui-1.8.21.custom.css" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?ver=1" rel="stylesheet" type="text/css"/>
<link href="/css/contents.css?ver=1" rel="stylesheet" type="text/css"/>
<script src="/js/jquery/jquery-1.7.2.min.js"></script>
<script src="/js/jquery/jquery-ui-1.8.21.custom.min.js"></script>
<script src="/js/jquery/plugin/jquery.jstree.js"></script>
<script src="/js/jquery/plugin/jquery.kmaform.js"></script>
<script src="/js/jquery/plugin/jquery.cookie.js"></script>
<script src="/js/jquery/plugin/jquery.form.js"></script>
<script src="/js/jquery/plugin/jquery.hotkeys.js"></script>
<script src="/js/jquery/plu

In [290]:
# 컬럼명 뽑아오기
col1 = soup.select(".list > thead > tr")[0].text.strip().split('\n')
col1.pop()
col1

['지점번호', '종료시각', '시작시각', '(현)지점명', '(현)주소', '위도', '경도', '노장해발고도(m)']

In [291]:
# '관측장비지상높이(m)'의 세부 컬럼명 추출
col2 = soup.select(".list > thead > tr")[1].text.strip().split('\n')
col2

['기압계', '기온계', '풍속계', '강우계']

In [292]:
asos_col = col1 + col2
asos_col

['지점번호',
 '종료시각',
 '시작시각',
 '(현)지점명',
 '(현)주소',
 '위도',
 '경도',
 '노장해발고도(m)',
 '기압계',
 '기온계',
 '풍속계',
 '강우계']

In [293]:
# 종료시간에 대한 컬럼은 필요없음. 전부 다 현재까지 운영중이여서
asos_col.pop(1)
asos_col

['지점번호',
 '시작시각',
 '(현)지점명',
 '(현)주소',
 '위도',
 '경도',
 '노장해발고도(m)',
 '기압계',
 '기온계',
 '풍속계',
 '강우계']

필드 데이터 뽑아오기

In [294]:
lst = soup.select(".list > tbody > tr")
lst

[<tr>
 <td> 90</td>
 <td>~</td>
 <td> 1968.01.01 00:00:00</td>
 <td> 속초</td>
 <td class="left"> 강원도 고성군 토성면 봉포리</td>
 <td> 38.25085</td>
 <td> 128.56473</td>
 <td> 17.53</td>
 <td> 18.73</td>
 <td> 1.7</td>
 <td> 10.0</td>
 <td class="last"> 1.4</td>
 </tr>, <tr>
 <td> 93</td>
 <td>~</td>
 <td> 2016.10.01 00:01:00</td>
 <td> 북춘천</td>
 <td class="left"> </td>
 <td> 37.94738</td>
 <td> 127.75443</td>
 <td> 95.78</td>
 <td> 96.78</td>
 <td> 1.5</td>
 <td> 10.0</td>
 <td class="last"> 1.4</td>
 </tr>, <tr>
 <td> 95</td>
 <td>~</td>
 <td> 1988.01.01 00:00:00</td>
 <td> 철원</td>
 <td class="left"> 강원도 철원군 갈말읍 군탄리</td>
 <td> 38.14787</td>
 <td> 127.3042</td>
 <td> 155.48</td>
 <td> 156.98</td>
 <td> 1.8</td>
 <td> 13.0</td>
 <td class="last"> 1.5</td>
 </tr>, <tr>
 <td> 98</td>
 <td>~</td>
 <td> 1998.02.01 00:00:00</td>
 <td> 동두천</td>
 <td class="left"> 경기도 동두천시 생연동</td>
 <td> 37.90188</td>
 <td> 127.0607</td>
 <td> 115.62</td>
 <td> 116.74</td>
 <td> 1.7</td>
 <td> 10.0</td>
 <td class="last"

In [295]:
# test
lst[0].text.split()

['90',
 '~',
 '1968.01.01',
 '00:00:00',
 '속초',
 '강원도',
 '고성군',
 '토성면',
 '봉포리',
 '38.25085',
 '128.56473',
 '17.53',
 '18.73',
 '1.7',
 '10.0',
 '1.4']

In [296]:
# test - 시작시간 합치기
lst[0].text.split()[2] +' '+ lst[0].text.split()[3]

'1968.01.01 00:00:00'

In [297]:
# test - 주소 추출
lst[0].select_one(".left").text.strip()

'강원도 고성군 토성면 봉포리'

In [298]:
# test
dd = lst[0].text.split()[:1]

In [299]:
# test
dd.append(lst[0].text.split()[2])  # 시작시간
dd.append(lst[0].text.split()[4])  # 지점명
dd.append(lst[0].select_one(".left").text.strip())  # 주소

In [300]:
# test
dd

['90', '1968.01.01', '속초', '강원도 고성군 토성면 봉포리']

In [301]:
# test - 리스트 합치기
dd + lst[0].text.split()[-7:]

['90',
 '1968.01.01',
 '속초',
 '강원도 고성군 토성면 봉포리',
 '38.25085',
 '128.56473',
 '17.53',
 '18.73',
 '1.7',
 '10.0',
 '1.4']

In [302]:
asos_data=[]

for a in range(len(lst)):
  dd = lst[a].text.split()[:1]
  dd.append(lst[a].text.split()[2])  # 시작시간
  dd.append(lst[a].text.split()[4])  # 지점명
  dd.append(lst[a].select_one(".left").text.strip())  # 주소
  data = dd + lst[a].text.split()[-7:]

  asos_data.append(data)

asos_data

[['90',
  '1968.01.01',
  '속초',
  '강원도 고성군 토성면 봉포리',
  '38.25085',
  '128.56473',
  '17.53',
  '18.73',
  '1.7',
  '10.0',
  '1.4'],
 ['93',
  '2016.10.01',
  '북춘천',
  '',
  '37.94738',
  '127.75443',
  '95.78',
  '96.78',
  '1.5',
  '10.0',
  '1.4'],
 ['95',
  '1988.01.01',
  '철원',
  '강원도 철원군 갈말읍 군탄리',
  '38.14787',
  '127.3042',
  '155.48',
  '156.98',
  '1.8',
  '13.0',
  '1.5'],
 ['98',
  '1998.02.01',
  '동두천',
  '경기도 동두천시 생연동',
  '37.90188',
  '127.0607',
  '115.62',
  '116.74',
  '1.7',
  '10.0',
  '1.0'],
 ['99',
  '2001.12.07',
  '파주',
  '경기도 파주시 문산읍 운천리',
  '37.88589',
  '126.76648',
  '30.59',
  '31.99',
  '1.7',
  '10.0',
  '1.0'],
 ['100',
  '1971.07.15',
  '대관령',
  '강원도 평창군 대관령면 횡계리',
  '37.67713',
  '128.71834',
  '772.43',
  '773.43',
  '1.7',
  '10.0',
  '1.4'],
 ['101',
  '1966.01.01',
  '춘천',
  '강원도 춘천시 우두동',
  '37.90262',
  '127.7357',
  '75.82',
  '77.05',
  '1.5',
  '10.0',
  '1.4'],
 ['102',
  '2000.11.01',
  '백령도',
  '인천광역시 옹진군 백령면 연화리',
  '37.97396',
  '124.7123

In [303]:
asos_region_df = pd.DataFrame(asos_data, columns= asos_col)
asos_region_df

,지점번호,시작시각,(현)지점명,(현)주소,위도,경도,노장해발고도(m),기압계,기온계,풍속계,강우계
0,90,1968.01.01,속초,강원도 고성군 토성면 봉포리,38.25085,128.56473,17.53,18.73,1.7,10.0,1.4
1,93,2016.10.01,북춘천,,37.94738,127.75443,95.78,96.78,1.5,10.0,1.4
2,95,1988.01.01,철원,강원도 철원군 갈말읍 군탄리,38.14787,127.3042,155.48,156.98,1.8,13.0,1.5
3,98,1998.02.01,동두천,경기도 동두천시 생연동,37.90188,127.0607,115.62,116.74,1.7,10.0,1.0
4,99,2001.12.07,파주,경기도 파주시 문산읍 운천리,37.88589,126.76648,30.59,31.99,1.7,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
90,285,1973.01.01,합천,경상남도 합천군 합천읍 합천리,35.56505,128.16994,26.72,33.2,1.5,10.0,0.5
91,288,1973.01.01,밀양,경상남도 밀양시 내이동,35.49147,128.74413,8.31,12.5,2.0,10.0,1.3
92,289,1972.03.30,산청,경상남도 산청군 산청읍 지리,35.413,127.8791,138.22,138.8,1.6,10.0,0.6
93,294,1972.01.24,거제,경상남도 거제시 신현읍 장평리,34.88818,128.60458,44.83,46.7,1.7,10.0,0.6


In [304]:
asos_region_df.rename(columns={'(현)지점명' : '지점명', '(현)주소' : '주소','노장해발고도(m)':'해발고도'}, inplace=True)
asos_region_df

,지점번호,시작시각,지점명,주소,위도,경도,해발고도,기압계,기온계,풍속계,강우계
0,90,1968.01.01,속초,강원도 고성군 토성면 봉포리,38.25085,128.56473,17.53,18.73,1.7,10.0,1.4
1,93,2016.10.01,북춘천,,37.94738,127.75443,95.78,96.78,1.5,10.0,1.4
2,95,1988.01.01,철원,강원도 철원군 갈말읍 군탄리,38.14787,127.3042,155.48,156.98,1.8,13.0,1.5
3,98,1998.02.01,동두천,경기도 동두천시 생연동,37.90188,127.0607,115.62,116.74,1.7,10.0,1.0
4,99,2001.12.07,파주,경기도 파주시 문산읍 운천리,37.88589,126.76648,30.59,31.99,1.7,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
90,285,1973.01.01,합천,경상남도 합천군 합천읍 합천리,35.56505,128.16994,26.72,33.2,1.5,10.0,0.5
91,288,1973.01.01,밀양,경상남도 밀양시 내이동,35.49147,128.74413,8.31,12.5,2.0,10.0,1.3
92,289,1972.03.30,산청,경상남도 산청군 산청읍 지리,35.413,127.8791,138.22,138.8,1.6,10.0,0.6
93,294,1972.01.24,거제,경상남도 거제시 신현읍 장평리,34.88818,128.60458,44.83,46.7,1.7,10.0,0.6


## 데이터 적재(MongoDB)

### 위도,경도 적재

In [305]:
# # DataFrame -> dict 로 변환
# import collections

# # 지점번호, 지점명,위도,경도만 적재
# asos_dict = asos_region_df.loc[:,['지점번호','지점명','위도','경도']].to_dict(orient='records')

In [306]:
# # json 복사해서 변수에 할당한거
# asos_dict = [{"index":0,"지점번호":"90","지점명":"속초","위도":"38.25085","경도":"128.56473"},{"index":1,"지점번호":"93","지점명":"북춘천","위도":"37.94738","경도":"127.75443"},{"index":2,"지점번호":"95","지점명":"철원","위도":"38.14787","경도":"127.3042"},{"index":3,"지점번호":"98","지점명":"동두천","위도":"37.90188","경도":"127.0607"},{"index":4,"지점번호":"99","지점명":"파주","위도":"37.88589","경도":"126.76648"},{"index":5,"지점번호":"100","지점명":"대관령","위도":"37.67713","경도":"128.71834"},{"index":6,"지점번호":"101","지점명":"춘천","위도":"37.90262","경도":"127.7357"},{"index":7,"지점번호":"102","지점명":"백령도","위도":"37.97396","경도":"124.71237"},{"index":8,"지점번호":"104","지점명":"북강릉","위도":"37.80456","경도":"128.85535"},{"index":9,"지점번호":"105","지점명":"강릉","위도":"37.75147","경도":"128.89099"},{"index":10,"지점번호":"106","지점명":"동해","위도":"37.50709","경도":"129.12433"},{"index":11,"지점번호":"108","지점명":"서울","위도":"37.57142","경도":"126.9658"},{"index":12,"지점번호":"112","지점명":"인천","위도":"37.47772","경도":"126.6249"},{"index":13,"지점번호":"114","지점명":"원주","위도":"37.33749","경도":"127.94659"},{"index":14,"지점번호":"115","지점명":"울릉도","위도":"37.48129","경도":"130.89864"},{"index":15,"지점번호":"119","지점명":"수원","위도":"37.25746","경도":"126.983"},{"index":16,"지점번호":"121","지점명":"영월","위도":"37.18126","경도":"128.45743"},{"index":17,"지점번호":"127","지점명":"충주","위도":"36.97045","경도":"127.9525"},{"index":18,"지점번호":"129","지점명":"서산","위도":"36.77658","경도":"126.4939"},{"index":19,"지점번호":"130","지점명":"울진","위도":"36.99176","경도":"129.41278"},{"index":20,"지점번호":"131","지점명":"청주","위도":"36.63924","경도":"127.44066"},{"index":21,"지점번호":"133","지점명":"대전","위도":"36.37198","경도":"127.37211"},{"index":22,"지점번호":"135","지점명":"추풍령","위도":"36.22025","경도":"127.99458"},{"index":23,"지점번호":"136","지점명":"안동","위도":"36.57293","경도":"128.70734"},{"index":24,"지점번호":"137","지점명":"상주","위도":"36.40837","경도":"128.15741"},{"index":25,"지점번호":"138","지점명":"포항","위도":"36.03201","경도":"129.38002"},{"index":26,"지점번호":"140","지점명":"군산","위도":"36.0053","경도":"126.76135"},{"index":27,"지점번호":"143","지점명":"대구","위도":"35.87797","경도":"128.65295"},{"index":28,"지점번호":"146","지점명":"전주","위도":"35.84092","경도":"127.11718"},{"index":29,"지점번호":"152","지점명":"울산","위도":"35.58237","경도":"129.33469"},{"index":30,"지점번호":"155","지점명":"창원","위도":"35.17019","경도":"128.57281"},{"index":31,"지점번호":"156","지점명":"광주","위도":"35.17294","경도":"126.89156"},{"index":32,"지점번호":"159","지점명":"부산","위도":"35.10468","경도":"129.03203"},{"index":33,"지점번호":"162","지점명":"통영","위도":"34.84541","경도":"128.43561"},{"index":34,"지점번호":"165","지점명":"목포","위도":"34.81732","경도":"126.38151"},{"index":35,"지점번호":"168","지점명":"여수","위도":"34.73929","경도":"127.74063"},{"index":36,"지점번호":"169","지점명":"흑산도","위도":"34.68719","경도":"125.45105"},{"index":37,"지점번호":"170","지점명":"완도","위도":"34.3959","경도":"126.70182"},{"index":38,"지점번호":"172","지점명":"고창","위도":"35.34824","경도":"126.599"},{"index":39,"지점번호":"174","지점명":"순천","위도":"35.0204","경도":"127.3694"},{"index":40,"지점번호":"177","지점명":"홍성","위도":"36.65759","경도":"126.68772"},{"index":41,"지점번호":"184","지점명":"제주","위도":"33.51411","경도":"126.52969"},{"index":42,"지점번호":"185","지점명":"고산","위도":"33.29382","경도":"126.16283"},{"index":43,"지점번호":"188","지점명":"성산","위도":"33.38677","경도":"126.8802"},{"index":44,"지점번호":"189","지점명":"서귀포","위도":"33.24616","경도":"126.5653"},{"index":45,"지점번호":"192","지점명":"진주","위도":"35.16378","경도":"128.04004"},{"index":46,"지점번호":"201","지점명":"강화","위도":"37.70739","경도":"126.44634"},{"index":47,"지점번호":"202","지점명":"양평","위도":"37.48863","경도":"127.49446"},{"index":48,"지점번호":"203","지점명":"이천","위도":"37.26399","경도":"127.48421"},{"index":49,"지점번호":"211","지점명":"인제","위도":"38.05986","경도":"128.16714"},{"index":50,"지점번호":"212","지점명":"홍천","위도":"37.6836","경도":"127.88043"},{"index":51,"지점번호":"216","지점명":"태백","위도":"37.17038","경도":"128.98929"},{"index":52,"지점번호":"217","지점명":"정선군","위도":"37.38149","경도":"128.6459"},{"index":53,"지점번호":"221","지점명":"제천","위도":"37.15928","경도":"128.19434"},{"index":54,"지점번호":"226","지점명":"보은","위도":"36.48761","경도":"127.73415"},{"index":55,"지점번호":"232","지점명":"천안","위도":"36.76217","경도":"127.29282"},{"index":56,"지점번호":"235","지점명":"보령","위도":"36.32724","경도":"126.55744"},{"index":57,"지점번호":"236","지점명":"부여","위도":"36.27242","경도":"126.92079"},{"index":58,"지점번호":"238","지점명":"금산","위도":"36.10563","경도":"127.48175"},{"index":59,"지점번호":"239","지점명":"세종","위도":"36.48522","경도":"127.24438"},{"index":60,"지점번호":"243","지점명":"부안","위도":"35.72961","경도":"126.71657"},{"index":61,"지점번호":"244","지점명":"임실","위도":"35.61203","경도":"127.28556"},{"index":62,"지점번호":"245","지점명":"정읍","위도":"35.56337","경도":"126.83904"},{"index":63,"지점번호":"247","지점명":"남원","위도":"35.4213","경도":"127.39652"},{"index":64,"지점번호":"248","지점명":"장수","위도":"35.65696","경도":"127.52031"},{"index":65,"지점번호":"251","지점명":"고창군","위도":"35.42661","경도":"126.697"},{"index":66,"지점번호":"252","지점명":"영광군","위도":"35.28366","경도":"126.47784"},{"index":67,"지점번호":"253","지점명":"김해시","위도":"35.22981","경도":"128.89075"},{"index":68,"지점번호":"254","지점명":"순창군","위도":"35.37131","경도":"127.1286"},{"index":69,"지점번호":"255","지점명":"북창원","위도":"35.22655","경도":"128.6726"},{"index":70,"지점번호":"257","지점명":"양산시","위도":"35.30737","경도":"129.0201"},{"index":71,"지점번호":"258","지점명":"보성군","위도":"34.76335","경도":"127.21226"},{"index":72,"지점번호":"259","지점명":"강진군","위도":"34.64457","경도":"126.78408"},{"index":73,"지점번호":"260","지점명":"장흥","위도":"34.68886","경도":"126.91951"},{"index":74,"지점번호":"261","지점명":"해남","위도":"34.55375","경도":"126.56907"},{"index":75,"지점번호":"262","지점명":"고흥","위도":"34.61826","경도":"127.27572"},{"index":76,"지점번호":"263","지점명":"의령군","위도":"35.32258","경도":"128.28812"},{"index":77,"지점번호":"264","지점명":"함양군","위도":"35.51138","경도":"127.74538"},{"index":78,"지점번호":"266","지점명":"광양시","위도":"34.9434","경도":"127.6914"},{"index":79,"지점번호":"268","지점명":"진도군","위도":"34.47296","경도":"126.25846"},{"index":80,"지점번호":"271","지점명":"봉화","위도":"36.94361","경도":"128.91449"},{"index":81,"지점번호":"272","지점명":"영주","위도":"36.87183","경도":"128.51688"},{"index":82,"지점번호":"273","지점명":"문경","위도":"36.62727","경도":"128.14879"},{"index":83,"지점번호":"276","지점명":"청송군","위도":"36.4351","경도":"129.04005"},{"index":84,"지점번호":"277","지점명":"영덕","위도":"36.53337","경도":"129.40926"},{"index":85,"지점번호":"278","지점명":"의성","위도":"36.3561","경도":"128.68864"},{"index":86,"지점번호":"279","지점명":"구미","위도":"36.13055","경도":"128.32056"},{"index":87,"지점번호":"281","지점명":"영천","위도":"35.97742","경도":"128.9514"},{"index":88,"지점번호":"283","지점명":"경주시","위도":"35.8174","경도":"129.2009"},{"index":89,"지점번호":"284","지점명":"거창","위도":"35.66739","경도":"127.9099"},{"index":90,"지점번호":"285","지점명":"합천","위도":"35.56505","경도":"128.16994"},{"index":91,"지점번호":"288","지점명":"밀양","위도":"35.49147","경도":"128.74413"},{"index":92,"지점번호":"289","지점명":"산청","위도":"35.413","경도":"127.8791"},{"index":93,"지점번호":"294","지점명":"거제","위도":"34.88818","경도":"128.60458"},{"index":94,"지점번호":"295","지점명":"남해","위도":"34.81662","경도":"127.92641"}]

In [307]:
# # csv로 저장
# asos_region_df.to_csv('ASOS_관측지점_위도,경도.csv',index=False)
# files.download('ASOS_관측지점_위도,경도.csv')

# # MongoDB로 저장
# import pymongo

# client = pymongo.MongoClient('mongodb://ec2-35-77-169-139.ap-northeast-1.compute.amazonaws.com:27017/')
# latitude = client.hsy.latitude
# latitude.insert_many(asos_dict)

### 시작시간 적재

In [308]:
# # 관측 시작 시간도 적재
# # 지점번호, 지점명, 시작시각
# asos_dict2 = asos_region_df.loc[:,['지점번호','지점명','시작시각']].to_dict(orient='records')
# startdt = client.hsy.startdt
# startdt.insert_many(asos_dict2)

In [309]:
# # csv로도 저장
# asos_region_df.loc[:,['지점번호','지점명','시작시각']].to_csv('ASOS_관측지점_시작날짜.csv',index=False)
# files.download('ASOS_관측지점_시작날짜.csv')

### [test] mongodb에서 데이터 불러오기

In [310]:
# # test
# QUERY = {
#     "지점번호":'90'
# }

# pd.DataFrame(latitude.find(QUERY))
# pd.DataFrame(latitude.find())

In [311]:
# # test - 윤지님이 적재하신 산불사건 불러와보기
# firedb = client.dbMYJ.users

# pd.DataFrame(firedb.find())

# # csv로 저장
# pd.DataFrame(firedb.find()).to_csv('mongo_db,화재사건,데이터(0701.ver).csv', index=False)
# files.download('mongo_db,화재사건,데이터(0701.ver).csv')

In [312]:
# test
# pd.DataFrame(startdt.find())

# ASOS 설립일 확인
- 기상데이터가 없었던 5,7,9,16,20,21,22번 확인
- 지점번호 : 217, 177, 106

In [313]:
asos_region_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지점번호    95 non-null     object
 1   시작시각    95 non-null     object
 2   지점명     95 non-null     object
 3   주소      95 non-null     object
 4   위도      95 non-null     object
 5   경도      95 non-null     object
 6   해발고도    95 non-null     object
 7   기압계     95 non-null     object
 8   기온계     95 non-null     object
 9   풍속계     95 non-null     object
 10  강우계     95 non-null     object
dtypes: object(11)
memory usage: 8.3+ KB


In [314]:
# 217 설립일 확인
asos_region_df[asos_region_df['지점번호'] == '217']

,지점번호,시작시각,지점명,주소,위도,경도,해발고도,기압계,기온계,풍속계,강우계
52,217,2010.08.06,정선군,강원도 정선군 정선읍 북실리,37.38149,128.6459,307.58,309.0,1.8,10.0,0.6


In [315]:
# 217 설립일이 정확한지 test
# forest_fires2('20100724','15','20100824','15','217','정선') -> error!
forest_fires2('20100825','15','20100825','15','217','정선')

[['정선', '217', '2010-08-25 15:00', '0.8', '230', '92', '20.2', '1.0']]

크롤링해온 데이터에는 '정선'의 경우 2010-08-06 부터 데이터가 있다고 되어있는데 실제로 숫자를 하나씩 넣어서 테스트해보면 2010-08-25 부터 데이터가 존재한다. 약간의 오차는 존재하는듯함.

In [316]:
# 177 설립일 확인
asos_region_df[asos_region_df['지점번호'] == '177']

,지점번호,시작시각,지점명,주소,위도,경도,해발고도,기압계,기온계,풍속계,강우계
40,177,2015.11.03,홍성,,36.65759,126.68772,27.74,28.7,1.5,10.0,1.5


In [317]:
# 177 설립일 정확도 확인
# forest_fires2('20141102','15','20151103','15','177','홍성') -> error!
forest_fires2('20151104','15','20151104','15','177','홍성')

[['홍성', '177', '2015-11-04 15:00', '1.6', '230', '35', '19.6', '']]

'홍성'의 경우 2015-11-03 부터 데이터가 존재한다고 되어있는데 실제로 확인을 해보면 2015-11-04 부터 데이터가 존재한다.

In [318]:
# 106 설립일 확인
asos_region_df[asos_region_df['지점번호'] == '106']

,지점번호,시작시각,지점명,주소,위도,경도,해발고도,기압계,기온계,풍속계,강우계
10,106,1992.05.01,동해,강원도 동해시 용정동,37.50709,129.12433,40.46,41.66,1.7,10.0,1.4


In [319]:
# 106 설립일 정확도 확인
# forest_fires2('19860201','15','19920201','15','106','동해') -> error! 
forest_fires2('19920202','15','19920202','15','106','동해')  # 근데 1992-04-10은 데이터 없음. 데이터 누락으로 추정

[['동해', '106', '1992-02-02 15:00', '', '', '', '5.0', '']]

'동해'의 경우 1992-05-01부터 데이터가 존재한다고 적혀있지만 그것보다 몇달전인 1992-02-02 부터 데이터가 존재한다. 
- 1992-04-09, 1992-04-11 다 데이터가 존재하는데 1992-04-10만 데이터가 없음. 데이터 누락으로 추정됨.

In [320]:
asos_region_df[asos_region_df['지점번호'] == '90']

,지점번호,시작시각,지점명,주소,위도,경도,해발고도,기압계,기온계,풍속계,강우계
0,90,1968.01.01,속초,강원도 고성군 토성면 봉포리,38.25085,128.56473,17.53,18.73,1.7,10.0,1.4


# 학습용 데이터(샘플3)
- 2005~2014 기간, 영동지방의 기상데이터 수집
- '실효습도' 에 관한 컬럼 추가
- 내가 맡은 연도 : 2013~ 2014
- 2013~ 2014 연도의 1월~4월, 10~12월 기상데이터
- 산불사건과 비교후 라벨링도 추가


In [321]:
# 자동화
datas = []

def plus_list(start,start_time,end,end_time,local,localname):
  global datas 
  datas = datas + forest_fires2(start,start_time,end,end_time,local,localname)

  return datas

In [322]:
# 속초
plus_list('20130101','00','20130131','23','90','속초')
plus_list('20130201','00','20130228','23','90','속초')
plus_list('20130301','00','20130331','23','90','속초')
plus_list('20130401','00','20130430','23','90','속초')
plus_list('20131001','00','20131031','23','90','속초')
plus_list('20131101','00','20131130','23','90','속초')
plus_list('20131201','00','20131231','23','90','속초')

plus_list('20140101','00','20140131','23','90','속초')
plus_list('20140201','00','20140228','23','90','속초')
plus_list('20140301','00','20140331','23','90','속초')
plus_list('20140401','00','20140430','23','90','속초')
plus_list('20141001','00','20141031','23','90','속초')
plus_list('20141101','00','20141130','23','90','속초')
plus_list('20141201','00','20141231','23','90','속초')

# 강릉
plus_list('20130101','00','20130131','23','105','강릉')
plus_list('20130201','00','20130228','23','105','강릉')
plus_list('20130301','00','20130331','23','105','강릉')
plus_list('20130401','00','20130430','23','105','강릉')
plus_list('20131001','00','20131031','23','105','강릉')
plus_list('20131101','00','20131130','23','105','강릉')
plus_list('20131201','00','20131231','23','105','강릉')

plus_list('20140101','00','20140131','23','105','강릉')
plus_list('20140201','00','20140228','23','105','강릉')
plus_list('20140301','00','20140331','23','105','강릉')
plus_list('20140401','00','20140430','23','105','강릉')
plus_list('20141001','00','20141031','23','105','강릉')
plus_list('20141101','00','20141130','23','105','강릉')
plus_list('20141201','00','20141231','23','105','강릉')

# 동해
plus_list('20130101','00','20130131','23','106','동해')
plus_list('20130201','00','20130228','23','106','동해')
plus_list('20130301','00','20130331','23','106','동해')
plus_list('20130401','00','20130430','23','106','동해')
plus_list('20131001','00','20131031','23','106','동해')
plus_list('20131101','00','20131130','23','106','동해')
plus_list('20131201','00','20131231','23','106','동해')

plus_list('20140101','00','20140131','23','106','동해')
plus_list('20140201','00','20140228','23','106','동해')
plus_list('20140301','00','20140331','23','106','동해')
plus_list('20140401','00','20140430','23','106','동해')
plus_list('20141001','00','20141031','23','106','동해')
plus_list('20141101','00','20141130','23','106','동해')
plus_list('20141201','00','20141231','23','106','동해')

# 태백
plus_list('20130101','00','20130131','23','216','태백')
plus_list('20130201','00','20130228','23','216','태백')
plus_list('20130301','00','20130331','23','216','태백')
plus_list('20130401','00','20130430','23','216','태백')
plus_list('20131001','00','20131031','23','216','태백')
plus_list('20131101','00','20131130','23','216','태백')
plus_list('20131201','00','20131231','23','216','태백')

plus_list('20140101','00','20140131','23','216','태백')
plus_list('20140201','00','20140228','23','216','태백')
plus_list('20140301','00','20140331','23','216','태백')
plus_list('20140401','00','20140430','23','216','태백')
plus_list('20141001','00','20141031','23','216','태백')
plus_list('20141101','00','20141130','23','216','태백')
data = plus_list('20141201','00','20141231','23','216','태백')

sample3_df = data_frame(data)

In [323]:
sample3_df[sample3_df['강수량'] != '']

,지역,지점번호,일시,풍속,풍향,습도,기온,강수량
12,속초,90,2013-01-01 12:00,1.2,140,57,1.1,0.0
24,속초,90,2013-01-02 00:00,1.2,320,72,-1.3,0.0
27,속초,90,2013-01-02 03:00,3.2,250,48,-3.1,0.0
333,속초,90,2013-01-14 21:00,2.3,320,35,2.3,0.0
336,속초,90,2013-01-15 00:00,1.4,250,97,-1.5,2.5
...,...,...,...,...,...,...,...,...
40320,태백,216,2014-12-16 09:00,2.9,250,56,-4.5,0.0
40413,태백,216,2014-12-20 06:00,3.4,230,76,1.2,0.1
40416,태백,216,2014-12-20 09:00,3.0,230,76,0.8,0.1
40419,태백,216,2014-12-20 12:00,5.3,230,50,-0.2,0.1


## 산불사건 라벨링

In [324]:
import pymongo

client = pymongo.MongoClient('mongodb://ec2-35-77-169-139.ap-northeast-1.compute.amazonaws.com:27017/')
firedb = client.dbMYJ.users

fire_df = pd.DataFrame(firedb.find())
fire_df

# # csv로 불러내기 -> csv로 불러내면 데이터 타입이 바뀜. object -> int 이런식으로
# fire_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/mongo_db,화재사건,데이터(0701.ver).csv')
# fire_df

,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,62be5e0852482b6dab131e1d,201406191500,목,201406191800,300,북부,강원,인제,남,남전,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,62be5e0852482b6dab131e1e,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,62be5e0852482b6dab131e1f,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,62be5e0852482b6dab131e20,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,62be5e0852482b6dab131e21,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11719,62be5e0852482b6dab134be4,191107022013,,,,대전,대전,중,대사,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11720,62be5e0852482b6dab134be5,191107022013,월,191101010300,1225,대전,대전,중,대사,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11721,62be5e0852482b6dab134be6,191106290400,,,,대전,대전,동,천,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11722,62be5e0852482b6dab134be7,191106290400,월,191101010000,1225,대전,대전,동,천,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0


In [325]:
# 2013, 2014년 사건만 추리기
event3_df = fire_df[fire_df['발생일시'] > '201300000000']
event3_df

,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,62be5e0852482b6dab131e1d,201406191500,목,201406191800,300,북부,강원,인제,남,남전,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,62be5e0852482b6dab131e1e,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,62be5e0852482b6dab131e1f,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,62be5e0852482b6dab131e20,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,62be5e0852482b6dab131e21,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,62be5e0852482b6dab1320e7,201301141530,월,201301141600,30,부산,부산,기장,장안,덕선,...,6.4,17.0,0.0,0.0,6.6,3.7,NNE,N,12,1.0
715,62be5e0852482b6dab1320e8,201301121640,토,201301121850,210,강원,강원,양양,강현,용호,...,4.7,49.0,0.0,0.0,3.3,5.4,SSE,WSW,13,0.2
716,62be5e0852482b6dab1320e9,201301111520,금,201301111620,100,전남,전남,해남,계곡,신평,...,4.6,24.0,0.0,0.0,3.9,2.8,N,NW,11,2.0
717,62be5e0852482b6dab1320ea,201301081440,화,201301081745,305,제주,제주,제주,추자,대서리,...,6.6,52.0,0.0,0.0,7.1,3.8,N,NNW,9,11.4


In [326]:
event3_df = event3_df.drop('_id', axis=1).sort_values(by='발생일시').reset_index(drop=True)
event3_df 

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,201301081200,화,201301081400,200,전남,전남,완도,약산,득암,산10-2,...,4.1,40.0,0.0,0.0,7.3,3.5,NNW,NNE,9,1.7
1,201301081440,화,201301081745,305,제주,제주,제주,추자,대서리,산160,...,6.6,52.0,0.0,0.0,7.1,3.8,N,NNW,9,11.4
2,201301111520,금,201301111620,100,전남,전남,해남,계곡,신평,산118-1,...,4.6,24.0,0.0,0.0,3.9,2.8,N,NW,11,2.0
3,201301121640,토,201301121850,210,강원,강원,양양,강현,용호,산88,...,4.7,49.0,0.0,0.0,3.3,5.4,SSE,WSW,13,0.2
4,201301141530,월,201301141600,30,부산,부산,기장,장안,덕선,산165-3,...,6.4,17.0,0.0,0.0,6.6,3.7,NNE,N,12,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,산66,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
715,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,산65,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
716,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,산70,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
717,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,산123-1,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5


In [327]:
# 컬럼복사
event3_df.columns

Index(['발생일시', '발생일시(요일)', '진화일시(년월일시분)', '진화소요시간(분)', '발생장소(관서)', '발생장소(시도)',
       '발생장소(시군구)', '발생장소(읍면동)', '발생장소(리)', '발생장소(지번)', '소유구분', '발생세부원인',
       '피해면적', '피해액', '산불위험지수(평균)', '산불위험지수(최대)', '평균기온', '실효습도', '상대습도',
       '최소습도', '최대풍속', '평균풍속', '최대풍향', '평균풍향', '강우결과일수', '최종강우량'],
      dtype='object')

In [328]:
event3_df2 = event3_df.loc[event3_df['발생장소(관서)'] == '강원',['발생일시','발생장소(시도)','발생장소(시군구)','발생장소(읍면동)','발생장소(리)','발생장소(지번)']]
event3_df2

,발생일시,발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번)
3,201301121640,강원,양양,강현,용호,산88
27,201302281210,강원,삼척,원덕,호산,산145-1
37,201303041227,강원,삼척,근덕,동막,산587-2
52,201303061510,강원,춘천,남산,산수,산143
58,201303081213,강원,양양,현남,하월천,산270
...,...,...,...,...,...,...
674,201405111636,강원,인제,남,부평,산93
676,201405131119,강원,고성,간성,광산,산54
694,201405261210,강원,삼척,근덕,교곡,산75-6
710,201406012145,강원,횡성,갑천,상대,산90-1


In [329]:
event3_df2['주소'] = event3_df2['발생장소(시도)'] + ' ' +  event3_df2['발생장소(시군구)'] + ' ' + event3_df2['발생장소(리)'] + '리 ' + event3_df2['발생장소(지번)']
event3_df2['주소']

3         강원 양양 용호리 산88
27     강원 삼척 호산리 산145-1
37     강원 삼척 동막리 산587-2
52       강원 춘천 산수리 산143
58      강원 양양 하월천리 산270
             ...       
674       강원 인제 부평리 산93
676       강원 고성 광산리 산54
694     강원 삼척 교곡리 산75-6
710     강원 횡성 상대리 산90-1
716       강원 인제 가아리 산70
Name: 주소, Length: 77, dtype: object

### 카카오 API 활용 (로컬 API)
- 영동지방 (속초, 강릉, 동해, 태백)의 사건만 추리기
- 카카오 API 이용해서 사건 발생지점과 가장 가까운 위도, 경도를 가진 관측지점을 선택해서 라벨링하기
- 사건이 일어난 해당 날짜의 시간대는 1, 사건과 관련없는 시간대는 0으로
- x : 경도
- y : 위도

In [330]:
# 카카오 API 세팅

KAKAO_API_KEY = "3644bda5f1ec337a30bc034d4e383a0a"
KAKAO_LOCAL_URL = "https://dapi.kakao.com/v2/local/search/address?query={}"

header = {
"Authorization": "KakaoAK {}".format(KAKAO_API_KEY)
}

longitude  = []
latitude = []

for query in list(event3_df2['주소']):
  response = requests.get(KAKAO_LOCAL_URL.format(query), headers=header)
  datas = response.json()

  try:
    longitude.append(datas['documents'][0]['address']['x'])  # 경도
    latitude.append(datas['documents'][0]['address']['y'] ) # 위도
  except:
    print('주소 error!')
    
    try:
      # 삼척의 위,경도
      pp = '삼척'
      re.search(pp, query).group()  # 주소에 '삼척' 이라는 단어가 안들어있으면 error! 뜨게끔
      longitude.append('129.16214')
      latitude.append('37.45003')

    except:
      try:
        # 양양의 위,경도
        pp = '양양'
        re.search(pp, query).group()
        longitude.append('128.56473')
        latitude.append('38.25085')
      except:
        try:
          # 홍천 팔봉리의 위,경도
          pp = '홍천'
          re.search(pp, query).group()
          longitude.append('127.70066')
          latitude.append('37.68614')
        except:
          try:
            # 강릉의 위,경도
            pp = '강릉'
            re.search(pp, query).group()
            longitude.append('128.89099')
            latitude.append('37.75147')
          except:
            try:
              # 태백
              pp = '태백'
              re.search(pp, query).group()
              longitude.append('128.98929')
              latitude.append('37.17038')
            except:
              try:
                # 속초
                pp = '속초'
                re.search(pp, query).group()
                longitude.append('128.56473')
                latitude.append('38.25085')
              except:
                try:
                  # 양구의 위,경도
                  pp = '양구'
                  re.search(pp, query).group()
                  longitude.append('127.98527')
                  latitude.append('38.09799')
                except:
                  # 원주 (강원도 원주시 관설동 산158-4)에서 제일 가까운 원주백운산 지점의 위,경도
                  pp = '원주'
                  re.search(pp, query).group()
                  longitude.append('127.9702')
                  latitude.append('37.2557')

event3_df2['longitude'] = longitude
event3_df2['latitude'] = latitude

주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!
주소 error!


In [331]:
# 불필요한 컬럼 삭제
event3_df2 = event3_df2.drop(['발생장소(시도)', '발생장소(시군구)', '발생장소(읍면동)', '발생장소(리)', '발생장소(지번)'], axis=1)
event3_df2

,발생일시,주소,longitude,latitude
3,201301121640,강원 양양 용호리 산88,128.613777359974,38.1287907875368
27,201302281210,강원 삼척 호산리 산145-1,129.16214,37.45003
37,201303041227,강원 삼척 동막리 산587-2,129.236261678975,37.3434599937119
52,201303061510,강원 춘천 산수리 산143,127.666645294348,37.7013790188893
58,201303081213,강원 양양 하월천리 산270,128.56473,38.25085
...,...,...,...,...
674,201405111636,강원 인제 부평리 산93,128.086554931329,37.9898009749704
676,201405131119,강원 고성 광산리 산54,128.40408761736,38.3762716291372
694,201405261210,강원 삼척 교곡리 산75-6,129.214861823516,37.3488560948734
710,201406012145,강원 횡성 상대리 산90-1,128.096537758693,37.4506449284038


### 제일 가까운 관측지점 찾기
- from scipy.spatial import distance 사용

In [332]:
latitude = client.hsy.latitude
latitude_df = pd.DataFrame(latitude.find())
latitude_df = latitude_df[(latitude_df['지점번호'] == '90') | (latitude_df['지점번호'] == '105') | (latitude_df['지점번호'] == '106') | (latitude_df['지점번호'] == '216')]
latitude_df

# # csv 로 -> 데이터 타입이 바뀜. object -> int 형으로
# latitude_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/mongo_db,관측지점,위경도(0701.ver).csv')
# latitude_df = latitude_df[(latitude_df['지점번호'] == 90) | (latitude_df['지점번호'] == 105) | (latitude_df['지점번호'] == 106) | (latitude_df['지점번호'] == 216)]
# latitude_df

,_id,지점번호,지점명,위도,경도
0,62be6fb4a82199005252108d,90,속초,38.25085,128.56473
9,62be6fb4a821990052521096,105,강릉,37.75147,128.89099
10,62be6fb4a821990052521097,106,동해,37.50709,129.12433
51,62be6fb4a8219900525210c0,216,태백,37.17038,128.98929


In [333]:
latitude_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 51
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     4 non-null      object
 1   지점번호    4 non-null      object
 2   지점명     4 non-null      object
 3   위도      4 non-null      object
 4   경도      4 non-null      object
dtypes: object(5)
memory usage: 192.0+ bytes


In [334]:
event3_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 3 to 716
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   발생일시       77 non-null     object
 1   주소         77 non-null     object
 2   longitude  77 non-null     object
 3   latitude   77 non-null     object
dtypes: object(4)
memory usage: 3.0+ KB


In [335]:
# 위, 경도 숫자 데이터로 바꾸기
latitude_df['위도'] = latitude_df['위도'].astype(float)
latitude_df['경도'] = latitude_df['경도'].astype(float)
event3_df2['longitude'] = event3_df2['longitude'].astype(float)
event3_df2['latitude'] = event3_df2['latitude'].astype(float)

In [336]:
from scipy.spatial import distance

a1_dist = []
a2_dist = []
a3_dist = []
a4_dist = []

for c,d in zip(event3_df2['latitude'],event3_df2['longitude']):
  a1 = tuple(latitude_df.iloc[0,3:5]) # 속초
  a2 = tuple(latitude_df.iloc[1,3:5]) # 강릉
  a3 = tuple(latitude_df.iloc[2,3:5]) # 동해
  a4 = tuple(latitude_df.iloc[3,3:5]) # 태백

  b = (c,d)

  # 각 지점마다의 유클리디안 거리 구하기
  a1_dist.append(distance.euclidean(a1, b))
  a2_dist.append(distance.euclidean(a2, b))
  a3_dist.append(distance.euclidean(a3, b))
  a4_dist.append(distance.euclidean(a4, b))

event3_df2['속초'] = a1_dist
event3_df2['강릉'] = a2_dist
event3_df2['동해'] = a3_dist
event3_df2['태백'] = a4_dist

In [337]:
event3_df2

,발생일시,주소,longitude,latitude,속초,강릉,동해,태백
3,201301121640,강원 양양 용호리 산88,128.613777,38.128791,0.131545,0.468207,0.804472,1.029350
27,201302281210,강원 삼척 호산리 산145-1,129.162140,37.450030,0.999105,0.405448,0.068450,0.328757
37,201303041227,강원 삼척 동막리 산587-2,129.236262,37.343460,1.128854,0.534495,0.198251,0.301582
52,201303061510,강원 춘천 산수리 산143,127.666645,37.701379,1.052841,1.225369,1.470576,1.425254
58,201303081213,강원 양양 하월천리 산270,128.564730,38.250850,0.000000,0.596512,0.930769,1.160890
...,...,...,...,...,...,...,...,...
674,201405111636,강원 인제 부평리 산93,128.086555,37.989801,0.544792,0.838998,1.144547,1.219172
676,201405131119,강원 고성 광산리 산54,128.404088,38.376272,0.203805,0.792118,1.128816,1.340387
694,201405261210,강원 삼척 교곡리 산75-6,129.214862,37.348856,1.111874,0.516712,0.182302,0.287639
710,201406012145,강원 횡성 상대리 산90-1,128.096538,37.450645,0.927110,0.849500,1.029341,0.935711


In [338]:
# 컬럼 복사 편하게 하려고 쓴것
event3_df2.columns

Index(['발생일시', '주소', 'longitude', 'latitude', '속초', '강릉', '동해', '태백'], dtype='object')

In [339]:
# 거리가 최소인 관측 지점 구하기
event3_df2 = event3_df2.reset_index(drop=True)  # loc를 사용하기위해 인덱스 리셋하기

for a in range(len(event3_df2.loc[:,'속초'])):
  m = event3_df2.loc[a,['속초', '강릉', '동해', '태백']].min(axis=0)
  m = str(m)

  event3_df2.loc[:,['속초', '강릉', '동해', '태백']] = event3_df2.loc[:,['속초', '강릉', '동해', '태백']].astype(str)

  # 문자열 포함여부 판단
  min_dist = event3_df2.loc[a,(event3_df2.loc[a,:].str.contains(m, case=False, regex=False) == True)].index[0]

  event3_df2.loc[a,'지점'] = min_dist

event3_df2.loc[:,'지점']

0     속초
1     동해
2     동해
3     속초
4     속초
      ..
72    속초
73    속초
74    동해
75    강릉
76    속초
Name: 지점, Length: 77, dtype: object

In [340]:
event3_df2

,발생일시,주소,longitude,latitude,속초,강릉,동해,태백,지점
0,201301121640,강원 양양 용호리 산88,128.613777,38.128791,0.1315450298093974,0.4682070316618219,0.8044724156000569,1.029349785294686,속초
1,201302281210,강원 삼척 호산리 산145-1,129.162140,37.450030,0.9991052899970044,0.4054483889473515,0.06845027173065532,0.32875712159585546,동해
2,201303041227,강원 삼척 동막리 산587-2,129.236262,37.343460,1.1288540292608757,0.5344948058994053,0.1982510522443959,0.3015819862641713,동해
3,201303061510,강원 춘천 산수리 산143,127.666645,37.701379,1.0528411549748564,1.225368950417254,1.4705757117376452,1.4252540038360488,속초
4,201303081213,강원 양양 하월천리 산270,128.564730,38.250850,0.0,0.5965115019846609,0.9307690893019548,1.160890440351719,속초
...,...,...,...,...,...,...,...,...,...
72,201405111636,강원 인제 부평리 산93,128.086555,37.989801,0.5447916939229278,0.8389978744538227,1.1445466257483514,1.2191723989780467,속초
73,201405131119,강원 고성 광산리 산54,128.404088,38.376272,0.20380520149319786,0.7921180505410758,1.1288161029062782,1.3403866792350283,속초
74,201405261210,강원 삼척 교곡리 산75-6,129.214862,37.348856,1.1118742702454196,0.5167116358946058,0.18230189192860605,0.28763929496087154,동해
75,201406012145,강원 횡성 상대리 산90-1,128.096538,37.450645,0.9271095574032036,0.8494999043075716,1.0293410209441578,0.9357109566804783,강릉


In [341]:
# 제일 가까운 관측지점 구했으니 위,경도 데이터는 삭제
event3_df2 = event3_df2.drop(['longitude', 'latitude', '속초', '강릉', '동해', '태백'], axis=1)

### 라벨링

In [342]:
sample3_df

,지역,지점번호,일시,풍속,풍향,습도,기온,강수량
0,속초,90,2013-01-01 00:00,6.1,250,43,-4.9,
1,속초,90,2013-01-01 01:00,5.5,250,42,-4.6,
2,속초,90,2013-01-01 02:00,4.3,270,41,-4.4,
3,속초,90,2013-01-01 03:00,3.5,250,45,-5.0,
4,속초,90,2013-01-01 04:00,2.7,290,53,-5.9,
...,...,...,...,...,...,...,...,...
40690,태백,216,2014-12-31 19:00,0.8,270,50,-4.8,
40691,태백,216,2014-12-31 20:00,1.8,270,38,-5.2,
40692,태백,216,2014-12-31 21:00,2.4,250,44,-5.7,
40693,태백,216,2014-12-31 22:00,3.1,250,41,-6.5,


In [343]:
event3_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   발생일시    77 non-null     object
 1   주소      77 non-null     object
 2   지점      77 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


#### 산불사건 데이터셋(event3_df2)의 발생일시 데이터 날짜파싱

In [344]:
import datetime

# 날짜형식으로 바꾸기
datetime_data = []

for i in event3_df2['발생일시']:
  date = i[:4] + '-' + i[4:6] + '-' + i[6:8] + ' ' + i[8:10] + ':' + i[10:]
  datetime_data.append(date)

event3_df2['발생일시'] = datetime_data
event3_df2['발생일시'] = pd.to_datetime(event3_df2['발생일시'])

In [345]:
# 산불사건 데이터 30분 이하는 버리고 30분 이상은 올리기
# ex) 16:40:00 -> 17:00:00, 15:10:00 -> 15:00:00
df22 = event3_df2.copy()  # 혹시 모르니 복사해서 사용

for b in df22.loc[(df22['발생일시'].dt.minute < 30),'발생일시']:
  df22.loc[(df22['발생일시'] == b),'발생일시'] = b.replace(minute=00)

for b in df22.loc[(df22['발생일시'].dt.minute >= 30),'발생일시']:
  c = b + datetime.timedelta(hours = 1)
  df22.loc[(df22['발생일시'] == b),'발생일시'] = c.replace(minute=00)

In [346]:
# 문자열로 파싱
df22['발생일시'] = df22['발생일시'].astype(str)

# 두 데이터프레임이 같은 형식을 띄도록 맞추기
sample3_df['일시'] = pd.to_datetime(sample3_df['일시'])
sample3_df['일시'] = sample3_df['일시'].astype(str)

In [347]:
lst1 = df22.loc[(df22['지점'] == '속초'),'발생일시']
lst2 = df22.loc[(df22['지점'] == '강릉'),'발생일시']
lst3 = df22.loc[(df22['지점'] == '동해'),'발생일시']
lst4 = df22.loc[(df22['지점'] == '태백'),'발생일시']

sample3_df['라벨'] = 0

# # 한번에 같이 for문 돌리니까 리스트 길이 짧은게 for 문 끝나면 다른 애들도 for문이 종료되버림. 
# # 각각 따로따로 for문 돌려야됨.
# for a,b,c,d in zip(lst1,lst2,lst3,lst4):
#   sample3_df.loc[(sample3_df['지역'] == '속초') & (sample3_df['일시'] == a) ,'라벨'] = 1
#   sample3_df.loc[(sample3_df['지역'] == '강릉') & (sample3_df['일시'] == b) ,'라벨'] = 1
#   sample3_df.loc[(sample3_df['지역'] == '동해') & (sample3_df['일시'] == c) ,'라벨'] = 1
#   sample3_df.loc[(sample3_df['지역'] == '태백') & (sample3_df['일시'] == d) ,'라벨'] = 1

for a in lst1:
  sample3_df.loc[(sample3_df['지역'] == '속초') & (sample3_df['일시'] == a) ,'라벨'] = 1

for b in lst2:
  sample3_df.loc[(sample3_df['지역'] == '강릉') & (sample3_df['일시'] == b) ,'라벨'] = 1

for c in lst3:
  sample3_df.loc[(sample3_df['지역'] == '동해') & (sample3_df['일시'] == c) ,'라벨'] = 1

for d in lst4:
  sample3_df.loc[(sample3_df['지역'] == '태백') & (sample3_df['일시'] == d) ,'라벨'] = 1


# # 데이터 저장
# sample3_df.to_csv('sample33333.csv',index=False)
# files.download('sample33333.csv')

In [348]:
len(sample3_df[sample3_df['라벨'] == 1])

57

In [349]:
sample3_df.loc[sample3_df['라벨'] == 1, '지역'].value_counts()

속초    35
태백    11
강릉     8
동해     3
Name: 지역, dtype: int64

## 실효습도

In [350]:
he_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/STCS_실효습도_20130101~20140430.csv', encoding='cp949')
he_df

,지점번호,지점명,시간,건조실효습도(%)
0,90,속초,2013-01-01 0:00,42.07
1,90,속초,2013-01-01 1:00,42.19
2,90,속초,2013-01-01 2:00,41.76
3,90,속초,2013-01-01 3:00,44.30
4,90,속초,2013-01-01 4:00,47.22
...,...,...,...,...
31765,216,태백,2014-04-30 19:00,65.63
31766,216,태백,2014-04-30 20:00,68.64
31767,216,태백,2014-04-30 21:00,69.70
31768,216,태백,2014-04-30 22:00,71.13


In [351]:
# sample3_df와 형식 맞추기
he_df['시간'] = pd.to_datetime(he_df['시간'])
he_df['시간'] = he_df['시간'].astype(str)
he_df.rename(columns={'시간' : '일시'},inplace=True)

In [352]:
# 산불데이터는 2014년 6월까지 존재하고 사이언스팀에서 요펑한 자료의 월은 1~4월, 10~12월 이였으니
# 2014년은 4월까지만 활용
sam = sample3_df[sample3_df['일시'] < '2014-05-01 00:00:00']
he = he_df.sort_values(by=['지점번호','일시'])  # 순서 뒤죽박죽 섞여있으니 정렬 후 사용

In [353]:
# 실효습도 데이터 합치기
test1 = sam[sam['지역'] == '속초']
test2 = he[he['지점명'] == '속초'].drop(['지점번호','지점명'], axis = 1)

test3 = sam[sam['지역'] == '강릉']
test4 = he[he['지점명'] == '강릉'].drop(['지점번호','지점명'], axis = 1)

test5 = sam[sam['지역'] == '동해']
test6 = he[he['지점명'] == '동해'].drop(['지점번호','지점명'], axis = 1)

test7 = sam[sam['지역'] == '태백']
test8 = he[he['지점명'] == '태백'].drop(['지점번호','지점명'], axis = 1)

test_12_df = pd.merge(test1, test2, on = '일시', how = 'outer')
test_34_df = pd.merge(test3, test4, on = '일시', how = 'outer')
test_56_df = pd.merge(test5, test6, on = '일시', how = 'outer')
test_78_df = pd.merge(test7, test8, on = '일시', how = 'outer')

test_all_df = pd.concat([test_12_df,test_34_df], axis=0)
test_all_df = pd.concat([test_all_df,test_56_df], axis=0)
test_all_df = pd.concat([test_all_df,test_78_df], axis=0)

In [354]:
# 컬럼복사용
test_all_df.columns

Index(['지역', '지점번호', '일시', '풍속', '풍향', '습도', '기온', '강수량', '라벨', '건조실효습도(%)'], dtype='object')

In [355]:
# 컬럼순서 변경 -> 라벨 컬럼이 맨 끝으로 가게끔
sample3_final_df = test_all_df.loc[:,['지역', '지점번호', '일시', '풍속', '풍향', '습도', '기온', '강수량', '건조실효습도(%)', '라벨']]
sample3_final_df

# # 데이터 저장
# sample3_final_df.to_csv('sample3(0705).csv', index=False)
# files.download('sample3(0705).csv')

,지역,지점번호,일시,풍속,풍향,습도,기온,강수량,건조실효습도(%),라벨
0,속초,90,2013-01-01 00:00:00,6.1,250,43,-4.9,,42.07,0
1,속초,90,2013-01-01 01:00:00,5.5,250,42,-4.6,,42.19,0
2,속초,90,2013-01-01 02:00:00,4.3,270,41,-4.4,,41.76,0
3,속초,90,2013-01-01 03:00:00,3.5,250,45,-5.0,,44.30,0
4,속초,90,2013-01-01 04:00:00,2.7,290,53,-5.9,,47.22,0
...,...,...,...,...,...,...,...,...,...,...
7960,태백,216,2014-04-30 19:00:00,0.4,0,87,10.8,,65.63,0
7961,태백,216,2014-04-30 20:00:00,0.0,0,88,10.8,,68.64,0
7962,태백,216,2014-04-30 21:00:00,0.2,0,91,10.3,,69.70,0
7963,태백,216,2014-04-30 22:00:00,0.3,0,94,9.4,,71.13,0


In [356]:
sample3_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31863 entries, 0 to 7964
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지역         31863 non-null  object 
 1   지점번호       31863 non-null  object 
 2   일시         31863 non-null  object 
 3   풍속         31863 non-null  object 
 4   풍향         31863 non-null  object 
 5   습도         31863 non-null  object 
 6   기온         31863 non-null  object 
 7   강수량        31863 non-null  object 
 8   건조실효습도(%)  31770 non-null  float64
 9   라벨         31863 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 2.7+ MB


In [357]:
test_all_df.loc[test_all_df['건조실효습도(%)'].isna(),['일시', '건조실효습도(%)']]

,일시,건조실효습도(%)
4625,2013-12-12 17:00:00,NaN
4649,2013-12-13 17:00:00,NaN
4673,2013-12-14 17:00:00,NaN
4697,2013-12-15 17:00:00,NaN
4721,2013-12-16 17:00:00,NaN
...,...,...
7088,2014-03-25 11:00:00,NaN
7089,2014-03-25 12:00:00,NaN
7090,2014-03-25 13:00:00,NaN
7091,2014-03-25 14:00:00,NaN


건조실효습도 데이터가 몇개 빠진게 있음.